In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

In [38]:
train = np.load('C:\Shivangi\college\Sem 5\Deep Learning\DL project/train_features.npy')
test = np.load('C:\Shivangi\college\Sem 5\Deep Learning\DL project/test_features.npy')
train10 = np.load('C:\Shivangi\college\Sem 5\Deep Learning\DL project/train10_features.npy')
test10 = np.load('C:\Shivangi\college\Sem 5\Deep Learning\DL project/test10_features.npy')

# #concatenate train and train10  
train = np.concatenate((train, train10), axis=0)
test = np.concatenate((test, test10), axis=0)

# write to npy
np.save('C:\Shivangi\college\Sem 5\Deep Learning\DL project/train_features_all.npy', train)
np.save('C:\Shivangi\college\Sem 5\Deep Learning\DL project/test_features_all.npy', test)

In [39]:
csv_data = pd.read_csv(r"C:\Shivangi\college\Sem 5\Deep Learning\DL project\zips\30_patients_zip1.csv")
print(csv_data.shape)
for i in tqdm(range(2, 11)):
    csv = pd.read_csv(f"C:\Shivangi\college\Sem 5\Deep Learning\DL project\zips/30_patients_zip{i}.csv")
    csv_data = pd.concat([csv_data, csv], axis=0)
    print(csv_data.shape)

(1301, 6)


  0%|          | 0/9 [00:00<?, ?it/s]

 67%|██████▋   | 6/9 [00:00<00:00, 51.29it/s]

(2732, 6)
(6692, 6)
(11303, 6)
(12701, 6)
(13953, 6)
(18106, 6)
(19735, 6)
(21254, 6)


100%|██████████| 9/9 [00:00<00:00, 23.64it/s]

(63026, 6)


In [40]:
csv_data.shape, train10.shape, test10.shape, train.shape, test.shape

((63026, 6), (41732, 2048), (40, 2048), (62998, 2048), (10661, 2048))

In [41]:
selected_patient_ids = [
    "098_S_4018", "098_S_4017", "116_S_1271", "031_S_0294", "031_S_4021", 
    "023_S_4020", "031_S_4024", "099_S_4022", "116_S_4010", "037_S_4028",
    "024_S_4084", "067_S_4782", "011_S_4827", "014_S_2185", "014_S_4401", 
    "022_S_6069", "041_S_4060", "041_S_4138", "041_S_4143", "041_S_4874",
    "011_S_0002", "011_S_0003", "011_S_0005", "011_S_0008", "022_S_0007", 
    "100_S_0015", "023_S_0030", "023_S_0031", "011_S_0016", "073_S_4393",
    '941_S_6499', '016_S_6931' ,'018_S_2155' ,'082_S_1119', '027_S_0835','116_S_1243'
]

train_patient_ids = selected_patient_ids[:30]  # First 30 for training
test_patient_ids = selected_patient_ids[30:]   # Last 6 for testing

In [42]:
# Filter the csv data into train and test and save
train_csv = csv_data[csv_data['patient_id'].isin(train_patient_ids)]
test_csv = csv_data[csv_data['patient_id'].isin(test_patient_ids)]

train_csv.to_csv('C:\Shivangi\college\Sem 5\Deep Learning\DL project/zips/train_labels_all.csv', index=False)
test_csv.to_csv('C:\Shivangi\college\Sem 5\Deep Learning\DL project/zips/test_labels_all.csv', index=False)

In [43]:
test.shape, test_csv.shape

((10661, 2048), (10661, 6))

In [45]:
# Filter the features into train and test and save
train_features = train[train_csv.index]
test_csv = test_csv.reset_index(drop=True)
test_features = test[test_csv.index]

np.save('C:\Shivangi\college\Sem 5\Deep Learning\DL project/train_features_all.npy', train_features)
np.save('C:\Shivangi\college\Sem 5\Deep Learning\DL project/test_features_all.npy', test_features)

In [53]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch

# ############################### Configuration ###################################
'''
Update paths according to your directory structure.
'''

TRAIN_FEATURES_PATH = r"C:\Shivangi\college\Sem 5\Deep Learning\DL project\train_features_all.npy"
TEST_FEATURES_PATH = r"C:\Shivangi\college\Sem 5\Deep Learning\DL project\test_features_all.npy"

TRAIN_CSV_PATH = r"C:\Shivangi\college\Sem 5\Deep Learning\DL project\zips\train_labels_all.csv"
TEST_CSV_PATH = r"C:\Shivangi\college\Sem 5\Deep Learning\DL project\zips\test_labels_all.csv"

ADAS_CSV_PATH = r"C:\Shivangi\college\Sem 5\Deep Learning\DL project\greymatter\text_data\ADAS_scores.csv"
MMSE_CSV_PATH = r"C:\Shivangi\college\Sem 5\Deep Learning\DL project\greymatter\text_data\MMSE_scores.csv"
BIOMARKER_CSV_PATH = r"C:\Shivangi\college\Sem 5\Deep Learning\DL project\greymatter\text_data\APOE.csv"

IMAGE_METADATA_CSV_PATH = r"F:\DL_DATASET\Cohort_4_MRI_Images_02Dec2024.csv"
DIAGNOSIS_CSV_PATH = r"F:\DL_DATASET\DXSUM_04Dec2024.csv"

TRAIN_OUTPUT_PATH = r"C:\Shivangi\college\Sem 5\Deep Learning\DL project\zips\train_merged.csv"
TEST_OUTPUT_PATH = r"C:\Shivangi\college\Sem 5\Deep Learning\DL project\zips\test_merged.csv"

selected_patient_ids = [
    "098_S_4018", "098_S_4017", "116_S_1271", "031_S_0294", "031_S_4021", 
    "023_S_4020", "031_S_4024", "099_S_4022", "116_S_4010", "037_S_4028",
    "024_S_4084", "067_S_4782", "011_S_4827", "014_S_2185", "014_S_4401", 
    "022_S_6069", "041_S_4060", "041_S_4138", "041_S_4143", "041_S_4874",
    "011_S_0002", "011_S_0003", "011_S_0005", "011_S_0008", "022_S_0007", 
    "100_S_0015", "023_S_0030", "023_S_0031", "011_S_0016", "073_S_4393",
    "941_S_6499", "016_S_6931", "018_S_2155", "082_S_1119", "027_S_0835", "116_S_1243"
]

train_patient_ids = selected_patient_ids[:30]
test_patient_ids = selected_patient_ids[30:]

# ############################### Functions ########################################

def load_features(features_path):
    if os.path.exists(features_path):
        features = np.load(features_path)
        print(f"Loaded features from {os.path.basename(features_path)}, shape: {features.shape}")
        return features
    else:
        raise FileNotFoundError(f"Features file not found: {features_path}")

def load_csv(csv_path):
    if os.path.exists(csv_path):
        df = pd.read_csv(csv_path)
        print(f"Loaded CSV {os.path.basename(csv_path)}, shape: {df.shape}")
        return df
    else:
        raise FileNotFoundError(f"CSV file not found: {csv_path}")

def main():
    # Load features (just to ensure matching rows, not to merge them)
    train_features = load_features(TRAIN_FEATURES_PATH)
    test_features = load_features(TEST_FEATURES_PATH)
    
    # Load CSV files
    train_csv = load_csv(TRAIN_CSV_PATH)
    test_csv = load_csv(TEST_CSV_PATH)
    
    # Check feature and CSV row counts
    if train_features.shape[0] != len(train_csv):
        raise ValueError(f"Train features ({train_features.shape[0]}) != train CSV rows ({len(train_csv)})")
    if test_features.shape[0] != len(test_csv):
        raise ValueError(f"Test features ({test_features.shape[0]}) != test CSV rows ({len(test_csv)})")

    # Add row_id for linking to features later
    train_csv['row_id'] = np.arange(len(train_csv))
    test_csv['row_id'] = np.arange(len(test_csv))

    # Add a 'set' column
    train_csv['set'] = 'train'
    test_csv['set'] = 'test'

    combined_df = pd.concat([train_csv, test_csv], axis=0).reset_index(drop=True)
    print(f"Combined train and test data, shape: {combined_df.shape}")

    # Rename columns for consistent keys: patient_id -> PTID, datetime -> VISDATE if they exist
    if 'patient_id' in combined_df.columns:
        combined_df.rename(columns={'patient_id': 'PTID'}, inplace=True)
    if 'datetime' in combined_df.columns:
        combined_df.rename(columns={'datetime': 'VISDATE'}, inplace=True)

    # Convert VISDATE to datetime and then to just date
    if 'VISDATE' in combined_df.columns:
        combined_df['VISDATE'] = pd.to_datetime(combined_df['VISDATE']).dt.date

    # Load image metadata
    image_metadata = load_csv(IMAGE_METADATA_CSV_PATH)
    image_metadata = image_metadata[['image_id', 'subject_id', 'mri_visit', 'mri_date', 'mri_type', 'mri_weighting', 'mri_acq_plane']]
    image_metadata['mri_date'] = pd.to_datetime(image_metadata['mri_date']).dt.date
    image_metadata.rename(columns={'subject_id':'PTID', 'mri_date':'VISDATE'}, inplace=True)

    # Merge image metadata
    merged_df = pd.merge(combined_df, image_metadata, on=['PTID','VISDATE'], how='left')
    print(f"Merged image metadata, shape: {merged_df.shape}")

    # Load ADAS, MMSE, Biomarker, Diagnosis
    adas_df = load_csv(ADAS_CSV_PATH)[['PTID', 'VISDATE', 'TOTAL13']]
    mmse_df = load_csv(MMSE_CSV_PATH)[['PTID', 'VISDATE', 'MMSCORE']]
    biomarker_df = load_csv(BIOMARKER_CSV_PATH)[['PTID', 'APTESTDT', 'GENOTYPE', 'APUSABLE']]
    diagnosis_df = load_csv(DIAGNOSIS_CSV_PATH)[['PTID', 'EXAMDATE', 'DIAGNOSIS']]

    # Process dates for merging
    adas_df['VISDATE'] = pd.to_datetime(adas_df['VISDATE']).dt.date
    mmse_df['VISDATE'] = pd.to_datetime(mmse_df['VISDATE']).dt.date
    biomarker_df['APTESTDT'] = pd.to_datetime(biomarker_df['APTESTDT']).dt.date
    biomarker_df.rename(columns={'APTESTDT':'VISDATE'}, inplace=True)
    diagnosis_df['EXAMDATE'] = pd.to_datetime(diagnosis_df['EXAMDATE']).dt.date
    diagnosis_df.rename(columns={'EXAMDATE':'VISDATE'}, inplace=True)

    # Merge ADAS
    merged_df = pd.merge(merged_df, adas_df, on=['PTID','VISDATE'], how='left')
    # Merge MMSE
    merged_df = pd.merge(merged_df, mmse_df, on=['PTID','VISDATE'], how='left')
    # Merge Biomarker
    merged_df = pd.merge(merged_df, biomarker_df, on=['PTID','VISDATE'], how='left')
    # Merge Diagnosis
    merged_df = pd.merge(merged_df, diagnosis_df, on=['PTID','VISDATE'], how='left')
    print(f"Merged ADAS, MMSE, Biomarker, Diagnosis, shape: {merged_df.shape}")

    # At this point, merged_df contains row_id, set, PTID, VISDATE, and all metadata.
    # No features added from npy (just row_id).

    # Separate back into train and test
    train_merged = merged_df[merged_df['set'] == 'train'].reset_index(drop=True)
    test_merged = merged_df[merged_df['set'] == 'test'].reset_index(drop=True)

    # Optionally handle missing values here if needed
    # Or just save directly
    # For large scale: no memory error since no large arrays

    # Save final train and test CSV files without large npy features
    train_merged.to_csv(TRAIN_OUTPUT_PATH, index=False)
    test_merged.to_csv(TEST_OUTPUT_PATH, index=False)
    print(f"Train merged saved to {TRAIN_OUTPUT_PATH}, shape: {train_merged.shape}")
    print(f"Test merged saved to {TEST_OUTPUT_PATH}, shape: {test_merged.shape}")

if __name__ == "__main__":
    main()


Loaded features from train_features_all.npy, shape: (52365, 2048)
Loaded features from test_features_all.npy, shape: (10661, 2048)
Loaded CSV train_labels_all.csv, shape: (52365, 6)
Loaded CSV test_labels_all.csv, shape: (10661, 6)
Combined train and test data, shape: (63026, 8)
Loaded CSV Cohort_4_MRI_Images_02Dec2024.csv, shape: (17283, 23)
Merged image metadata, shape: (861198, 13)
Loaded CSV ADAS_scores.csv, shape: (15480, 15)
Loaded CSV MMSE_scores.csv, shape: (13800, 53)
Loaded CSV APOE.csv, shape: (2758, 7)
Loaded CSV DXSUM_04Dec2024.csv, shape: (14333, 41)
Merged ADAS, MMSE, Biomarker, Diagnosis, shape: (861198, 18)
Train merged saved to C:\Shivangi\college\Sem 5\Deep Learning\DL project\zips\train_merged.csv, shape: (783419, 18)
Test merged saved to C:\Shivangi\college\Sem 5\Deep Learning\DL project\zips\test_merged.csv, shape: (77779, 18)


In [54]:
print("done")

done
